In [2]:
import cobrakbase
from reframed import *

kbase = cobrakbase.KBaseAPI()

cobrakbase 0.2.7


In [3]:
# # Running FBA/SteadyCom on merged electro community model
model1 = from_cobrapy(kbase.get_from_ws('Acetobacterium_MES1.electrosynthesis.edited', 93204))
model1.id = 'm1'
model1.compartments.e0.external = True
model2 = from_cobrapy(kbase.get_from_ws('Sulfurospirillum_MES13.autotroph.edited', 93204))
model2.id = 'm2'
model2.compartments.e0.external = True
model3 = from_cobrapy(kbase.get_from_ws('Desulfovibrio_MES4.autotroph.edited.v2', 93204))
model3.id = 'm3'
model3.compartments.e0.external = True

for rxn_id in model1.reactions:
    if 'EX_' in rxn_id:
        rxn = model1.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE
for rxn_id in model2.reactions:
    if 'EX_' in rxn_id:
        rxn = model2.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE
for rxn_id in model3.reactions:
    if 'EX_' in rxn_id:
        rxn = model3.reactions[rxn_id]
        rxn.reaction_type = ReactionType.EXCHANGE
        
electroCommunity = Community('electro.community',[model1,model2,model3])

media = kbase.get_from_ws('CO2_minimal', 93204) #  we download the media from KBase
media_const = media.get_media_constraints() #  get constraints 

for r in electroCommunity.merged_model.get_exchange_reactions():
    rxn = electroCommunity.merged_model.reactions[r]
    if r[5:] in media_const: #  if constraint in media we assign bounds
        lb, ub = media_const[r[5:]]
        rxn.set_flux_bounds(lb, ub)
    else: #  otherwise set to excretion only
        rxn.set_flux_bounds(0, 1000)

print('FBA')
res = FBA(electroCommunity.merged_model) #  test if new media works
print(res)

print('SteadyCom')
sol = SteadyCom(electroCommunity)
print(sol)

FBA
Objective: 0.13415949482528114
Status: Optimal

SteadyCom
Community growth: 0.1337890625
m1	0.8999076500420199
m2	0.00024233331552488835
m3	0.0998500166424553
